In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import stem
import math
import operator

In [2]:
# carregando os dados
url = "https://raw.githubusercontent.com/LiviaCavalcanti/RecInfo/master/lab02/results.csv"
original_data = pd.read_csv(url)
N_TOTAL_DOCUMENTS = original_data.shape[0]
original_data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [19]:
from gensim.models import Word2Vec
model = Word2Vec.load('pt.bin')

In [87]:
def wmdistance(news1, news2):
    n1 = news2embeddings(news1)
    n2 = news2embeddings(news2)
    
    
    vocab_len = len(set(n1.keys())) + len(set(n2.keys()))
    docset1 = set(n1.keys())
    docset2 = set(n2.keys())

    # Compute distance matrix.
    distance_matrix = np.zeros((vocab_len, vocab_len))
    t1 = 0
    for i in n1.keys():
        t1+=1
        t2=0
        for j in n2.keys():
            t2+=1
            # Compute Euclidean distance between word vectors.
            math.sqrt(np.sum((n1[i] - n2[j])**2))
            distance_matrix[t1, t2] = math.sqrt(np.sum((n1[i] - n2[j])**2))
    return distance_matrix

In [76]:
distance_matrix = np.zeros((2, 2))

distance_matrix[0,1]

0.0

In [88]:
wmdistance(0,1)

sociedade
rubens
paiva
facínoras
facínoras
mataram
sociedade
rubens
paiva
facínoras
facínoras
mataram


/home/livia/Documentos/miniconda3/envs/ri/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 28.81206573, 27.06368628, 28.83975037,
         0.        ,  0.        ,  0.        ],
       [ 0.        , 28.81206573,  0.        , 14.85481547, 20.81787773,
         0.        ,  0.        ,  0.        ],
       [ 0.        , 27.06368628, 14.85481547,  0.        , 19.27237897,
         0.        ,  0.        ,  0.        ],
       [ 0.        , 28.83975037, 20.81787773, 19.27237897,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

In [89]:
def similarity(news_id):
    for i in original_data.index:
        wmdistance(news_id, i)
    

In [92]:
similarity(0)

/home/livia/Documentos/miniconda3/envs/ri/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [91]:
def news2embeddings(news_id, size=10):
    words = re.split(r'\W+', original_data.loc[0].title)
    
    embed_words = {}
    count2limit = 0
    for word in words:
        
        w = word.lower()
        
        if w not in stopwords_complement:
            try:
                embed_words[w] = model[w]
                count2limit += 1
            except:
                continue
        if count2limit == size:
            return embed_words
                
        
    return embed_words

In [3]:
# gerando tokens
array_df_token = []
original_data.title.apply(lambda news: array_df_token.append( re.split(r'\W+', news)))
array_df_token[0]

['',
 'A',
 'sociedade',
 'foi',
 'Rubens',
 'Paiva',
 'não',
 'os',
 'facínoras',
 'que',
 'o',
 'mataram',
 '']

In [4]:

# para o first time run
nltk.download('stopwords')

# removendo stopwords
default_stopwords = set(stopwords.words('portuguese'))
stopwords_complement = ["", "neste","és", "é", "somos", "sois", "são", "era", "eras", "éreis", "foste", "fostes", "foras",	"fôreis",	"serás", "sereis", "serão",	"serias", "seríeis", "sejas", "sejais", "fosses", "fôsseis", "fores", "fordes", "sê", "sejas", "sejais", "ser", "seres", "sermos",	"serdes",	"serem", "sobre", "é"]

stopwords_complement.extend(default_stopwords)



[nltk_data] Downloading package stopwords to /home/livia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stopwords_complement.extend(default_stopwords)


In [6]:
no_stopwords_news = []
for word_array in array_df_token:
    no_stopwords_news.append(list(filter(lambda x : x.lower() not in stopwords_complement, word_array)))

In [7]:
filtered_news = []

for e in no_stopwords_news:
    aux_word = []
    for word in e:
        aux_word.append(word.lower())
    filtered_news.append(aux_word[:10])

In [8]:
filtered_news[0:5]


[['sociedade', 'rubens', 'paiva', 'facínoras', 'mataram'],
 ['justiça',
  'suspende',
  'decisão',
  'proibia',
  'forças',
  'armadas',
  'celebrarem',
  'golpe',
  '1964'],
 ['governo', 'bolsonaro', 'prega', 'negacionismo', 'histórico', 'ditadura'],
 ['pais', 'gabo', 'perceberam', 'filho', 'mentiroso'],
 ['rádios',
  'canadenses',
  'banem',
  'músicas',
  'michael',
  'jackson',
  'programação']]

In [20]:
model["lei"]

/home/livia/Documentos/miniconda3/envs/ri/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-6.5462875e-01, -2.2935629e+00, -1.4184085e+00,  1.8673456e+00,
       -2.7797916e+00, -1.0738591e+00, -1.7973077e-01, -1.1639605e+00,
        1.0856268e+00,  1.8858245e-01, -2.2553296e-01,  2.3803563e+00,
       -4.5980744e-02,  7.3853351e-02, -1.0235522e+00, -5.9012938e-01,
        1.8113707e+00, -6.4247268e-01,  4.2772263e-01,  8.7607521e-01,
       -7.4397832e-01, -2.1257985e+00, -1.9487043e-01,  9.9959485e-02,
       -1.3591160e+00, -8.4675229e-01, -1.2846813e+00, -1.8125669e+00,
       -2.8817327e+00,  3.1197064e+00, -4.6457262e+00,  2.4174795e-01,
       -2.9926053e-01,  1.9427787e+00,  1.2972488e-01, -4.6086481e-01,
        1.1106607e+00, -1.1886199e-01, -1.1663800e+00,  6.9122076e-01,
       -8.4158637e-02,  2.0012600e+00, -4.0435207e-01,  1.2587290e+00,
       -6.0827559e-01, -7.0740506e-02,  7.0606381e-01,  2.7059095e+00,
        8.8259661e-01, -7.7127415e-01, -2.4471928e-01, -1.0108689e+00,
       -1.3424310e-01, -6.6520691e-01, -2.3377411e+00, -1.1457791e+00,
      

In [ ]:
model.similarity("similar", "semelhante")